In [1]:
!pip install bip-utils pycryptodome

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 872.5/872.5 kB 24.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.4/589.4 kB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 71.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.2/249.2 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.6/150.6 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 370.6/370.6 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 43.2 MB/s eta 0:00:00
  Created wheel for crcmod: filename=crcmod-1.7-cp311-cp311-linux_x86_64.whl size=31656 sha256=528170ce7b15e88c9dcad9dfb2d188fbdc676205eafe6e780ed4f9682892fa87
  Stored in directory: /root/.cache/pip

In [2]:
import os
import hashlib
import hmac
from bip_utils import Bip39MnemonicGenerator, Bip39EntropyGenerator, Bip39SeedGenerator
from Crypto.Hash import SHAKE256, keccak
from binascii import hexlify, unhexlify

**Step 1: Entropy and Mnemonic Phrase (BIP39)**

In [5]:
entropy = Bip39EntropyGenerator(256).Generate()
mnemonic = Bip39MnemonicGenerator().FromEntropy(entropy)
print("Mnemonic phasrse:")
print(mnemonic)


Mnemonic phasrse:
more mistake airport cake icon pet runway risk jealous soldier march cancel lonely focus inflict fuel diamond frost enable busy lawsuit square minor crime


**Step 2: Master Seed Derivation**



In [11]:
# Manual genration of master seed
import hashlib
import unicodedata

mnemonic_str = str(mnemonic)  # Ensure it's a string
passphrase = ""               # User-defined if needed

# Normalize the mnemonic (NFKD form)
mnemonic_bytes = unicodedata.normalize("NFKD", mnemonic_str).encode("utf-8")
salt_bytes = unicodedata.normalize("NFKD", "mnemonic" + passphrase).encode("utf-8")

# Perform PBKDF2-HMAC-SHA512 with 2048 iterations
master_seed = hashlib.pbkdf2_hmac("sha512", mnemonic_bytes, salt_bytes, 2048, dklen=64)

print("Master Seed (512 bits):", master_seed.hex())



Master Seed (512 bits): 0f56a309a93c816ff1df67c5ba141dd5e28c24785c9ace891c8409a5b3039bc86c03c1b77d95c2402912c4112306d2c8e55a4e75bad893499f315515b79e8cb7


In [6]:
# Automatic genration of master seed
# Optional passphrase
passphrase = ""  # Can be user-defined

# Derive master seed (512 bits)
seed_bytes = Bip39SeedGenerator(mnemonic).Generate(passphrase)

print("Master Seed (hex):", seed_bytes.hex())


Master Seed (hex): 0f56a309a93c816ff1df67c5ba141dd5e28c24785c9ace891c8409a5b3039bc86c03c1b77d95c2402912c4112306d2c8e55a4e75bad893499f315515b79e8cb7


In [7]:
# Split the 512-bit seed into components
dilithium_seed = seed_bytes[:32]         # 256 bits
mdecc_seed = seed_bytes[32:48]           # 128 bits
chain_code = seed_bytes[48:64]           # 128 bits

print("Dilithium Seed:", dilithium_seed.hex())
print("mdECC Seed:", mdecc_seed.hex())
print("Chain Code:", chain_code.hex())


Dilithium Seed: 0f56a309a93c816ff1df67c5ba141dd5e28c24785c9ace891c8409a5b3039bc8
mdECC Seed: 6c03c1b77d95c2402912c4112306d2c8
Chain Code: e55a4e75bad893499f315515b79e8cb7


**Step 3: Key Generation**

Dilithium5 Key Generation

In [8]:
def simulate_dilithium_keygen(seed: bytes):
    shake = SHAKE256.new()
    shake.update(seed)
    key_material = shake.read(2592 + 4864)
    public_key = key_material[:2592]
    private_key = key_material[2592:]
    return public_key, private_key

dilithium_pub, dilithium_priv = simulate_dilithium_keygen(dilithium_seed)

print("Dilithium Public Key (2592 bytes):", dilithium_pub[:32].hex(), "...")
print("Dilithium Private Key (4864 bytes):", dilithium_priv[:32].hex(), "...")


Dilithium Public Key (2592 bytes): 5b1dbf2d1e27f253f68b344cd92bd9fbb7f6ebaa9bd6591e1391f3863d17d2ce ...
Dilithium Private Key (4864 bytes): fda6af5b360bfef653d4da67ff22604e5ac884da43099ad7e20dc3b5b6ddb317 ...


mdECC Key Generation

In [9]:
def simulate_ecc_keypair(seed: bytes, curve_id: str, key_size: int):
    shake = SHAKE256.new()
    shake.update(seed + curve_id.encode())
    sk = int.from_bytes(shake.read(key_size), 'big')
    pk = hashlib.sha256(shake.read(key_size)).digest()
    return sk, pk

# Simulate mdECC for 3 curves
ecc_keys = {}
for name, size in [("P-521", 66), ("Curve448", 56), ("E-521", 66)]:
    sk, pk = simulate_ecc_keypair(mdecc_seed, name, size)
    ecc_keys[name] = {"private": sk, "public": pk}

print("P-521 Public Key:", ecc_keys["P-521"]["public"].hex())
print("Curve448 Public Key:", ecc_keys["Curve448"]["public"].hex())
print("E-521 Public Key:", ecc_keys["E-521"]["public"].hex())


P-521 Public Key: 4cc95cf717465f46978d138fc1434a6dbd3f26fbebb14688d80f0678f927ee85
Curve448 Public Key: 77abbc9426a52d4dff8bc1f34f370c9efb32922bbc427a79d990aa9231061ebc
E-521 Public Key: d1482185986344faf03184778a1cc17cb2650fc33aba43d8bb8965eadcff03e6


**Step 4: BlackChain Address Derivation**

In [10]:
def keccak256(data: bytes) -> bytes:
    k = keccak.new(digest_bits=256)
    k.update(data)
    return k.digest()

def derive_blackchain_address(dilithium_pub, ecc_keys, chain_code):
    # Combine all parts
    combined = (
        dilithium_pub +
        ecc_keys["P-521"]["public"] +
        ecc_keys["Curve448"]["public"] +
        ecc_keys["E-521"]["public"] +
        chain_code
    )
    h = keccak256(combined)
    raw_address = h[-20:]

    # EIP-55 checksum
    addr_hex = raw_address.hex()
    checksum = keccak256(addr_hex.encode()).hex()
    checksummed = "0x"
    for i, c in enumerate(addr_hex):
        if c in "abcdef" and int(checksum[i], 16) >= 8:
            checksummed += c.upper()
        else:
            checksummed += c
    return checksummed

address = derive_blackchain_address(dilithium_pub, ecc_keys, chain_code)
print("BlackChain Address:", address)


BlackChain Address: 0xf594D71d54a31f0e8Ea4CAE77294162b2F974665
